In [185]:
!pip3 install ipywidgets

In [186]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from six import iteritems

In [177]:
!pip3 install openpyxl

     -------------------------------------- 242.2/242.2 KB 1.9 MB/s eta 0:00:00


In [128]:
all_profile_info = {}

In [187]:
class Helper:
    @staticmethod
    def printDict(_dict):
        for key, value in iteritems(_dict):
            print(key,':',value)

    @staticmethod
    def printListDic(all_exp):
        for _dict in all_exp:
            printDict(_dict)
            print()
            
    @staticmethod
    def login(browser):
        browser.get('https://www.linkedin.com/uas/login')
        file = open('config.txt')
        lines = file.readlines()
        username = lines[0]
        password = lines[1]

        elementID = browser.find_element_by_id('username')
        elementID.send_keys(username)
        elementID = browser.find_element_by_id('password')
        elementID.send_keys(password)
        elementID.submit()
        
    @staticmethod
    def get_browser(logged_in=True):
        browser = webdriver.Chrome('driver/chromedriver.exe')
        if logged_in:
            Helper.login(browser)
        return browser
    
    @staticmethod
    def scroll(browser):
        SCROLL_PAUSE_TIME = 2
        last_height = browser.execute_script("return document.body.scrollHeight")
        for i in range(3):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
         
    @staticmethod   
    def get_profile_link_and_scroll(browser, url):
        browser.get(url)
        Helper.scroll(browser)
        
    @staticmethod
    def get_profile_id_from_linkedin_url(url):
        url = str(url)
        index = url.find('.com')
        return url[index+4:]
    
    @staticmethod
    def get_profile_url_from_id(_id):
        LINKEDIN_BASE_URL = 'https://www.linkedin.com'
        _id = str(_id)
        
        url = LINKEDIN_BASE_URL + _id
        return url
    
    @staticmethod
    def setBroswerAndGetSoup(browser, profile_url):
        browser.get(profile_url)
        Helper.scroll(browser)
        src = browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup
    
    @staticmethod
    def is_from_IIITA(all_edu_exp):
        IIITA_School_Company = 'https://www.linkedin.com/company/497317/'
        IIITA = 'https://www.linkedin.com/search/results/all/?keywords=IIIT+Allahabad'
        for each_edu_exp in all_edu_exp:
            if each_edu_exp['URL ID'] == IIITA_School_Company:
                return True
            if each_edu_exp['URL ID'] == IIITA:
                return True
        return False
    

In [188]:
class InfoScraper:
    def __init__(self, profile_url, logged_in_browser):
        self.profile_id = Helper.get_profile_id_from_linkedin_url(profile_url)
        self.profile_url = profile_url
        self.browser = logged_in_browser
        self.bs = self.setBroswerAndGetSoup()
        self.basicProfileInfo = self.getBasicProfileInfo()
        self.all_experiences = self.getAllExperieceInfo()
        self.all_educations = self.getAllEducationInfo()
        self.is_faculty = Helper.is_from_IIITA(self.all_experiences)
        self.is_student = Helper.is_from_IIITA(self.all_educations)
        self.profile_status = self.connectionStatus()
        
#     def __init__(self, json_item):
        

    def setBroswerAndGetSoup(self):
        self.browser.get(self.profile_url)
        Helper.scroll(self.browser)
        src = self.browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup

    def refreshStatus(self):
        self.bs = self.setBroswerAndGetSoup()
        self.profile_status = self.connectionStatus()
        
    def connectionStatus(self):
        bs = self.bs
        actions_div = bs.find('div', {'class': 'pvs-profile-actions'})
        if actions_div is None:
            return "Error"
        actions = actions_div.findAll('div', {'class': 'entry-point'})
        if ' to connect' in str(actions_div):
            return "Not Connected"
        if 'Pending' in str(actions_div):
            return "Request Pending"
        if actions is None or len(actions) == 0:
            return "Unknown"
        return "Connected"

    def getProfileName(self, bs):
        name_div = bs.find('div', {'class': 'mt2 relative'})
        name_loc = name_div.find_all('div')
        name = name_loc[0].find('h1').get_text().strip()
        return str(name)

    def getProfileLocation(self, bs):
        name_div = bs.find('div', {'class': 'mt2 relative'})
        name_loc = name_div.find_all('div')
        if len(name_loc) == 0:
            return 'None'
        loc = name_loc[-1].find('span').get_text().strip()
        return str(loc)

    def getBasicProfileInfo(self):
        bs = self.bs
        info = {}
        info['Name'] = self.getProfileName(bs)
        info['Location'] = self.getProfileLocation(bs)
        return info
    
    def getExpURLID(self, bs):
        ids = []
        href_classes = bs.findAll('a', {'data-field': 'experience_company_logo'})
        for href_class in href_classes:
            if href_class is None:
                continue
            exp_url = href_class.attrs.get('href', None)
            if exp_url is None:
                continue
            ids.append(exp_url)
        return ids

    def getExpTitle(self, all_exp_spans):
        if len(all_exp_spans) < 2:
            return 'None'
        job_title = all_exp_spans[1].get_text().strip()
        return str(job_title)

    def getExpCompany(self, all_exp_spans):
        if len(all_exp_spans) < 4:
            return 'None'
        double_text = str(all_exp_spans[3].get_text().strip())
        strip_index = len(double_text)//2
        company = double_text[0:strip_index].split(' · ')[0]
        return str(company)

    def getExpTimePeriod(self, all_exp_spans):
        if len(all_exp_spans) < 7:
            return 'None'
        time_period = all_exp_spans[6].get_text().strip().split(' · ')[0]
        return str(time_period)

    def getExpInfo(self, exp):
        info = {}
        all_exp_spans = exp.find_all('span')
        job_title = self.getExpTitle(all_exp_spans)
        company = self.getExpCompany(all_exp_spans)
        time_period = self.getExpTimePeriod(all_exp_spans)
        
        info['Job Title'] = job_title
        info['Company'] = company
        info['Time Period'] = time_period
        return info
        
    def getExperienceSetions(self, bs):
        profile_sections = bs.findAll('section')
        exp_profile_section = None
        for profile_section in profile_sections:
            x = profile_section.find('div', {'id': 'experience'})
            if x is not None:
                exp_profile_section = profile_section
                break
        if exp_profile_section is None:
            return {}
        exp_sections = exp_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
        return exp_sections

    def getAllExperieceInfo(self):
        bs = self.bs
        all_experiences = []
        company_ids = self.getExpURLID(bs)
        exp_sections = self.getExperienceSetions(bs)
        for exp in exp_sections:
            all_experiences.append(self.getExpInfo(exp))
        for i in range(len(all_experiences)):
            if i >= len(company_ids):
                all_experiences[i]['URL ID'] = 'None'
                continue
            all_experiences[i]['URL ID'] = str(company_ids[i])
        return all_experiences

    def getEduSchoolID(self, edu):
        href_class = edu.findAll('a')
        if href_class is None or len(href_class) == 0:
            return 'None'
        school_url = href_class[0].attrs.get('href', None)
        if school_url is None:
            return 'None'
        return str(school_url)

    def getEduSchool(self, all_edu_spans):
        if len(all_edu_spans) < 2:
            return 'None'
        job_title = all_edu_spans[1].get_text().strip()
        return str(job_title)

    def getEduCourse(self, all_edu_spans):
        if len(all_edu_spans) < 4:
            return 'None'
        course_and_discipline = all_edu_spans[4].get_text().strip().split(',')
        if len(course_and_discipline) < 1:
            return 'None'
        course = course_and_discipline[0]
        return str(course)

    def getEduDiscipline(self, all_edu_spans):
        if len(all_edu_spans) < 4:
            return 'None'
        line_text = all_edu_spans[4].get_text()
        course_and_discipline = line_text.strip().split(',')
        if len(course_and_discipline) < 2:
            return 'None'
        ind = line_text.find(', ')
        discipline = line_text[ind+1:]
        return str(discipline)

    def getEduTimePeriod(self, all_edu_spans):
        if len(all_edu_spans) < 8:
            return 'None'
        time_period = all_edu_spans[7].get_text().strip()
        return str(time_period)

    def getEduInfo(self, edu):
        info = {}
        # print(edu)
        all_edu_spans = edu.find_all('span')
        school_url_id = self.getEduSchoolID(edu)
        school = self.getEduSchool(all_edu_spans)
        course = self.getEduCourse(all_edu_spans)
        discipline = self.getEduDiscipline(all_edu_spans)
        time_period = self.getEduTimePeriod(all_edu_spans)
        info['URL ID'] = school_url_id
        info['Insititution'] = school
        info['Course'] = course
        info['Discipline'] = discipline
        info['Time Period'] = time_period
        return info
        
    def getEducationSetions(self, bs):
        profile_sections = bs.findAll('section')
        edu_profile_section = None
        for profile_section in profile_sections:
            x = profile_section.find('div', {'id': 'education'})
            if x is not None:
                edu_profile_section = profile_section
                break
        if edu_profile_section is None:
            return {}
        edu_sections = edu_profile_section.find_all('div', {'class': 'display-flex flex-column full-width align-self-center'})
        return edu_sections

    def getAllEducationInfo(self):
        bs = self.bs
        all_educations = []
        edu_sections = self.getEducationSetions(bs)
        for edu in edu_sections:
            all_educations.append(self.getEduInfo(edu))
        return all_educations

In [189]:
browser = Helper.get_browser()

c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [132]:
# profile_info = InfoScraper('https://www.linkedin.com/in/abhi4gupta/', browser)
# profile_info = InfoScraper('https://www.linkedin.com/in/dr-sunny-sharma-64887918/', browser)
profile_info_2 = InfoScraper('https://www.linkedin.com/in/anupam-agrawal-a822b011/', browser)
# print(profile_info.all_experiences)

In [190]:
profile_info_2.is_student

True

In [191]:
all_profile_info[profile_info_2.profile_id] = profile_info_2

In [192]:
print(all_profile_info['/in/anupam-agrawal-a822b011/'].is_faculty)

True


In [193]:
profile_urls = [
    'https://www.linkedin.com/in/anupam-agrawal-a822b011/',
    'https://www.linkedin.com/in/shivam-gupta-52302516b/',
    'https://www.linkedin.com/in/isha-goel30/',
    'https://www.linkedin.com/in/richik-bhattacharjee/',
    'https://www.linkedin.com/in/sougata-das-4132a616b/',
    'https://www.linkedin.com/in/abhi4gupta/',
    'https://www.linkedin.com/in/dr-sunny-sharma-64887918/',
    'https://www.linkedin.com/in/sankarsan-bhattacheryya-003526173/',
    'https://www.linkedin.com/in/soumabrata-bhattacharya-9b190a1a9/',
    'https://www.linkedin.com/in/arun-gangwar-14a428202/',
    'https://www.linkedin.com/in/avijit-agarwal-06b495148/',
    'https://www.linkedin.com/in/sanyam-garg-22a487135/',
    'https://www.linkedin.com/in/prateek-m-058795114/',
    'https://www.linkedin.com/in/anish-goswami-49a3b7185/',
    'https://www.linkedin.com/in/pranal-maheshwari-31a850192/',
    'https://www.linkedin.com/in/abhigyansharma29/',
    'https://www.linkedin.com/in/blackdchase/',
    'https://www.linkedin.com/in/raunak-rathour-84a7b71aa/',
    'https://www.linkedin.com/in/utkarsh--priyam/',
]

In [194]:
for profile_url in profile_urls:
    profile_id = Helper.get_profile_id_from_linkedin_url(profile_url)
    profile_info = None
    if profile_id not in all_profile_info:
        profile_info = InfoScraper(profile_url, browser)
        all_profile_info[profile_id] = profile_info
    else:
        profile_info = all_profile_info[profile_id]
    print(profile_id, '\t, Student :', profile_info.is_student, '\t, Faculty :', profile_info.is_faculty)

/in/anupam-agrawal-a822b011/ 	, Student : True 	, Faculty : True
/in/shivam-gupta-52302516b/ 	, Student : False 	, Faculty : False
/in/isha-goel30/ 	, Student : False 	, Faculty : False
/in/richik-bhattacharjee/ 	, Student : False 	, Faculty : False
/in/sougata-das-4132a616b/ 	, Student : False 	, Faculty : False
/in/abhi4gupta/ 	, Student : True 	, Faculty : False
/in/dr-sunny-sharma-64887918/ 	, Student : False 	, Faculty : True
/in/sankarsan-bhattacheryya-003526173/ 	, Student : False 	, Faculty : False
/in/soumabrata-bhattacharya-9b190a1a9/ 	, Student : False 	, Faculty : False
/in/arun-gangwar-14a428202/ 	, Student : False 	, Faculty : False
/in/avijit-agarwal-06b495148/ 	, Student : False 	, Faculty : False
/in/sanyam-garg-22a487135/ 	, Student : False 	, Faculty : False
/in/prateek-m-058795114/ 	, Student : True 	, Faculty : False
/in/anish-goswami-49a3b7185/ 	, Student : True 	, Faculty : False
/in/pranal-maheshwari-31a850192/ 	, Student : True 	, Faculty : False
/in/abhigyansh

In [195]:
print(all_profile_info)

{'/in/anupam-agrawal-a822b011/': <__main__.InfoScraper object at 0x000001F6FD76A4E0>, '/in/shivam-gupta-52302516b/': <__main__.InfoScraper object at 0x000001F6FDA2BE48>, '/in/isha-goel30/': <__main__.InfoScraper object at 0x000001F6FE397BE0>, '/in/richik-bhattacharjee/': <__main__.InfoScraper object at 0x000001F6FE5CC438>, '/in/sougata-das-4132a616b/': <__main__.InfoScraper object at 0x000001F6FE8FA1D0>, '/in/abhi4gupta/': <__main__.InfoScraper object at 0x000001F6FE748C88>, '/in/dr-sunny-sharma-64887918/': <__main__.InfoScraper object at 0x000001F6FEAB1F60>, '/in/sankarsan-bhattacheryya-003526173/': <__main__.InfoScraper object at 0x000001F6FEE309E8>, '/in/soumabrata-bhattacharya-9b190a1a9/': <__main__.InfoScraper object at 0x000001F6FEF9D4E0>, '/in/arun-gangwar-14a428202/': <__main__.InfoScraper object at 0x000001F6FF26D630>, '/in/avijit-agarwal-06b495148/': <__main__.InfoScraper object at 0x000001F6FF37EB70>, '/in/sanyam-garg-22a487135/': <__main__.InfoScraper object at 0x000001F6FF

In [196]:
class OtherProfileInfo:
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
        
    def getPeopleAlsoViewedSection(self, bs):
        profile_sections = bs.findAll('section')
        req_profile_section = None
        for profile_section in profile_sections:
            if 'People also viewed' in str(profile_section):
                return profile_section
        return None

    def getNewProfileIDs(self, profile, profilesQueued):
        profilesID = []
        bs = self.setBroswerAndGetSoup(profile)
        people_also_viewed_section = self.getPeopleAlsoViewedSection(bs)
        if people_also_viewed_section is None:
            return None
        all_links = people_also_viewed_section.findAll('a')
        for link in all_links:
            userID = link.get('href')
            if((userID is not None) and (userID not in profilesQueued) and (userID not in visitedProfiles)):
                profilesID.append(userID)
        return profilesID
    
    def setBroswerAndGetSoup(self, profile):
        self.browser.get(profile)
        src = self.browser.page_source
        soup = BeautifulSoup(src, 'lxml')
        return soup


In [197]:
other_profile = OtherProfileInfo(browser)
visitedProfiles = []
profilesQueued = []
other_profile.getNewProfileIDs('https://www.linkedin.com/in/isha-goel30/', profilesQueued)

In [198]:
class MyProfile:
    LINKEDIN_BASE_URL = 'https://www.linkedin.com'
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
        self.followers = self.getFollowersList()
        self.iiit_student, self.iiit_faculty = self.getIIITList(self.followers)
        
    def getFollowersList(self):
        followers_url = 'https://www.linkedin.com/feed/followers/'
        browser.get(followers_url)
        time.sleep(2)
        src = browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        lis = []
        all_followers = bs.find('div', {'class': 'feed-following'})
        if all_followers is None:
            return lis
        for each_follower in all_followers.findAll('a'):
            follower_id = each_follower.get('href')
            if follower_id is None:
                continue
            lis.append(follower_id)
            profile_info = InfoScraper('https://www.linkedin.com'+follower_id, browser)
            all_profile_info[profile_info.profile_id] = profile_info
        return list(set(list(lis)))
    
    def getIIITList(self, users):
        stud = []
        fac = []
        for user in users:
            print(user)
            if user in all_profile_info:
                if all_profile_info[user].is_student:
                    stud.append(user)
                if all_profile_info[user].is_faculty:
                    fac.append(user)
            else:
                print('Not in dict')
        return stud, fac
    
    def send_connection_request_with_text(self, profile_url, text):
        self.browser.get(profile_url)
        time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        
        actions_divs = bs.findAll('div', {'class': 'pvs-profile-actions'})
        if len(actions_divs) == 0:
            return
        ids = []
        for actions_div in actions_divs:
            buttons = actions_div.findAll('button')
            if len(buttons) == 0:
                continue
            _id = None
            for button in buttons:
                cnnt = button.get('aria-label')
                if cnnt is None:
                    continue
                if ' to connect' in str(cnnt):
                    _id = button.get('id')
                    break
            if _id is None:
                continue
            ids.append(_id)
        if len(ids) == 0:
            return
        self.browser.find_element_by_id(ids[-1]).click()
        time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Add a note':
                _id = x.get('id')
                break
        if _id is None:
            return
        self.browser.find_element_by_id(_id).click()
        time.sleep(1)
        INV_REQUEST = text
        elementID = browser.find_element_by_id('custom-message')
        elementID.send_keys(INV_REQUEST)
        
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        _id = None
        for x in bs.find_all('button'):
            if x.get('aria-label') == 'Send now':
                _id = x.get('id')
                break
        if _id is None:
            return
        self.browser.find_element_by_id(_id).click()
   
    def send_text_to_connection(self, profile_url, text):
        self.browser.get(profile_url)
        # time.sleep(1)
        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')
        x = bs.find('a', {'class': 'message-anywhere-button pvs-profile-actions__action artdeco-button'})
        if x is None:
            return
        link = x.get('href')
        if link is None:
            return
        full_link = 'https://www.linkedin.com' + link
        self.browser.get(full_link)
        time.sleep(1)

        src = self.browser.page_source
        bs = BeautifulSoup(src, 'lxml')

        message_text = self.browser.find_element_by_css_selector('div.msg-form__contenteditable > p')
        message_text.send_keys(text)
        message_text.submit()
        
        time.sleep(3)

    
    def sendAnnouncement(self, users, inbox_text, connection_req_text):
        for user in users:
            profile_url = Helper.get_profile_url_from_id(user)
            profile_info = None
            if user not in all_profile_info:
                profile_info = InfoScraper(profile_url, self.browser)
                all_profile_info[user] = profile_info
            else:
                profile_info = all_profile_info[user]
                # profile_info.refreshStatus() # Refresh Status
            if profile_info is None:
                return
            status = profile_info.profile_status
            print(status)
            if status == 'Connected':
                self.send_text_to_connection(profile_url, inbox_text)
            elif status == 'Not Connected':
                self.send_connection_request_with_text(profile_url, connection_req_text)
        return

In [199]:
my_profile = MyProfile(browser)

/in/harshit-narula-2a82ba162/
/in/prateek-m-058795114/
/in/harsh-arora-3a8a0a1a8/
/in/mitali-seth-4649801a1/
/in/utkarsh-pandey-95015b170/
/in/raunak-rathour-84a7b71aa/
/in/shubham-kumar-7a75851a2/
/in/abhirup-das-740981178/
/in/sankarsan-bhattacheryya-003526173/
/in/utkarsh--priyam/
/in/aniket-mallick-75572416b/
/in/abhi4gupta/
/in/kapil-tomar-a2b31118b/
/in/sougata-das-4132a616b/
/in/shubhankar-raj-457b30231/
/in/shivam-gupta-52302516b/
/in/sambuddha-koner-98732b206/
/in/isha-goel30/
/in/siddhi-singh-8653801b1/
/in/richik-bhattacharjee/


In [143]:
lis = my_profile.followers
print(my_profile.getIIITList(lis))
# text_list = [
#     # '/in/abhi4gupta/',
#     '/in/avijit-agarwal-06b495148/',
# ]
# my_profile.sendAnnouncement(text_list,'Hi!','Please accept my request')

/in/harshit-narula-2a82ba162/
/in/prateek-m-058795114/
/in/harsh-arora-3a8a0a1a8/
/in/mitali-seth-4649801a1/
/in/utkarsh-pandey-95015b170/
/in/raunak-rathour-84a7b71aa/
/in/subharoop-ghosh-943669150/
/in/shubham-kumar-7a75851a2/
/in/abhirup-das-740981178/
/in/sankarsan-bhattacheryya-003526173/
/in/utkarsh--priyam/
/in/aniket-mallick-75572416b/
/in/abhi4gupta/
/in/kapil-tomar-a2b31118b/
/in/sougata-das-4132a616b/
/in/shivam-gupta-52302516b/
/in/sambuddha-koner-98732b206/
/in/isha-goel30/
/in/siddhi-singh-8653801b1/
/in/richik-bhattacharjee/
(['/in/prateek-m-058795114/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/', '/in/abhi4gupta/'], [])


In [144]:
print(my_profile.iiit_student, my_profile.iiit_faculty)

['/in/prateek-m-058795114/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/', '/in/abhi4gupta/'] []


In [145]:
# print(profile_urls)

In [ ]:
test_followers = [Helper.get_profile_id_from_linkedin_url(each_profile) for each_profile in profile_urls ]
print(test_followers)

['/in/anupam-agrawal-a822b011/', '/in/shivam-gupta-52302516b/', '/in/isha-goel30/', '/in/richik-bhattacharjee/', '/in/sougata-das-4132a616b/', '/in/abhi4gupta/', '/in/dr-sunny-sharma-64887918/', '/in/sankarsan-bhattacheryya-003526173/', '/in/soumabrata-bhattacharya-9b190a1a9/', '/in/arun-gangwar-14a428202/', '/in/avijit-agarwal-06b495148/', '/in/sanyam-garg-22a487135/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/']


In [ ]:
iiit_student, iiit_faculty = my_profile.getIIITList(test_followers)
print(iiit_student, iiit_faculty)

/in/anupam-agrawal-a822b011/
/in/shivam-gupta-52302516b/
/in/isha-goel30/
/in/richik-bhattacharjee/
/in/sougata-das-4132a616b/
/in/abhi4gupta/
/in/dr-sunny-sharma-64887918/
/in/sankarsan-bhattacheryya-003526173/
/in/soumabrata-bhattacharya-9b190a1a9/
/in/arun-gangwar-14a428202/
/in/avijit-agarwal-06b495148/
/in/sanyam-garg-22a487135/
/in/prateek-m-058795114/
/in/anish-goswami-49a3b7185/
/in/pranal-maheshwari-31a850192/
/in/abhigyansharma29/
/in/blackdchase/
/in/raunak-rathour-84a7b71aa/
/in/utkarsh--priyam/
['/in/anupam-agrawal-a822b011/', '/in/abhi4gupta/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/'] ['/in/anupam-agrawal-a822b011/', '/in/dr-sunny-sharma-64887918/']


In [ ]:
print(all_profile_info['/in/abhi4gupta/'].all_experiences)

[{'Job Title': 'SDE intern', 'Company': 'Uber', 'Time Period': 'Jan 2022 - Present', 'URL ID': 'https://www.linkedin.com/company/1815218/'}, {'Job Title': 'software developer', 'Company': 'Citi', 'Time Period': 'May 2021 - Jul 2021', 'URL ID': 'https://www.linkedin.com/company/11448/'}, {'Job Title': 'Lead Technical Writer', 'Company': 'Talentio Solutions India Private Limited', 'Time Period': 'Jun 2020 - Jul 2020', 'URL ID': 'https://www.linkedin.com/company/3184270/'}, {'Job Title': 'Teaching Assistance', 'Company': 'Prayaas IIITA', 'Time Period': 'Jan 2020 - Jun 2020', 'URL ID': 'https://www.linkedin.com/company/14545739/'}, {'Job Title': 'Student Trainee', 'Company': 'Dakshana Foundation', 'Time Period': 'Jul 2017 - Jun 2018', 'URL ID': 'https://www.linkedin.com/company/15611786/'}]


In [ ]:
print(all_profile_info['/in/abhi4gupta/'].all_educations)

[{'URL ID': 'https://www.linkedin.com/company/497317/', 'Insititution': 'Indian Institute Of Information Technology', 'Course': 'Bachelor of Technology - BTech', 'Discipline': ' Information Technology', 'Time Period': '2018 - 2022'}, {'URL ID': 'https://www.linkedin.com/company/14515894/', 'Insititution': 'Jawahar Navodaya Vidyalaya (JNV)', 'Course': '6th to 12th', 'Discipline': ' General Studies', 'Time Period': '2010 - 2017'}]


In [ ]:
list(all_profile_info.keys())

['/in/anupam-agrawal-a822b011/',
 '/in/shivam-gupta-52302516b/',
 '/in/isha-goel30/',
 '/in/richik-bhattacharjee/',
 '/in/sougata-das-4132a616b/',
 '/in/abhi4gupta/',
 '/in/dr-sunny-sharma-64887918/',
 '/in/sankarsan-bhattacheryya-003526173/',
 '/in/soumabrata-bhattacharya-9b190a1a9/',
 '/in/arun-gangwar-14a428202/',
 '/in/avijit-agarwal-06b495148/',
 '/in/sanyam-garg-22a487135/',
 '/in/prateek-m-058795114/',
 '/in/anish-goswami-49a3b7185/',
 '/in/pranal-maheshwari-31a850192/',
 '/in/abhigyansharma29/',
 '/in/blackdchase/',
 '/in/raunak-rathour-84a7b71aa/',
 '/in/utkarsh--priyam/',
 '/in/harsh-arora-3a8a0a1a8/',
 '/in/sambuddha-koner-98732b206/',
 '/in/kapil-tomar-a2b31118b/',
 '/in/mitali-seth-4649801a1/',
 '/in/harshit-narula-2a82ba162/',
 '/in/shubham-kumar-7a75851a2/',
 '/in/siddhi-singh-8653801b1/',
 '/in/utkarsh-pandey-95015b170/',
 '/in/aniket-mallick-75572416b/',
 '/in/abhirup-das-740981178/',
 '/in/subharoop-ghosh-943669150/']

In [ ]:
# test_text_followers = [Helper.get_profile_id_from_linkedin_url(each_profile) for each_profile in text_profile_urls ]

In [126]:
test_list = [
    '/in/raunak-rathour-84a7b71aa/',
]

my_profile.sendAnnouncement(test_list,'Ok','Plese accept my request!')

Connected


c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:114: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


In [132]:
class ProfilesByFilter(object):
    def __init__(self, logged_in_browser):
        self.browser = logged_in_browser
    
    def getFilteredFaculty(self):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if prof_obj.is_faculty is True:
                ids.append(prof_id)
        return ids
    
    def getFilteredBranch(self, course):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if course == 'bachelor':
                for edu in prof_obj.all_educations:
                    if edu['Course'] and ('bachelor' in edu['Course'].lower() or 'btech' in  edu['Course'].lower()):
                        ids.append(prof_id)
            if course == 'master':
                for edu in prof_obj.all_educations:
                    if edu['Course'] and ('master' in edu['Course'].lower() or 'mtech' in  edu['Course'].lower() or 'mba' in  edu['Course'].lower()):
                        ids.append(prof_id)
        return ids
    
    def getFilteredStudent(self):
        ids = []
        for prof_id, prof_obj in iteritems(all_profile_info):
            if prof_obj.is_student is True:
                ids.append(prof_id)
        return ids
    
    def getFiltered(self, course = None, student = False, faculty = False):
        if faculty is True:
            return self.getFilteredFaculty()
        if student is True:
            return self.getFilteredStudent()
        if course is not None:
            return self.getFilteredBranch(course)
        return []
    

In [133]:
filtered_profiles = ProfilesByFilter(browser)
print(filtered_profiles.getFiltered(student = True))

['/in/anupam-agrawal-a822b011/', '/in/abhi4gupta/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/', '/in/ayushraj0309/']


In [134]:
# print(all_profile_info.keys())

In [137]:
print(filtered_profiles.getFiltered(course = 'bachelor'))

['/in/isha-goel30/', '/in/richik-bhattacharjee/', '/in/sougata-das-4132a616b/', '/in/abhi4gupta/', '/in/sankarsan-bhattacheryya-003526173/', '/in/soumabrata-bhattacharya-9b190a1a9/', '/in/arun-gangwar-14a428202/', '/in/sanyam-garg-22a487135/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/', '/in/sambuddha-koner-98732b206/', '/in/kapil-tomar-a2b31118b/', '/in/mitali-seth-4649801a1/', '/in/harshit-narula-2a82ba162/', '/in/shubham-kumar-7a75851a2/', '/in/siddhi-singh-8653801b1/', '/in/utkarsh-pandey-95015b170/', '/in/aniket-mallick-75572416b/', '/in/abhirup-das-740981178/', '/in/subharoop-ghosh-943669150/', '/in/harsh-arora-3a8a0a1a8/', '/in/ayushraj0309/']


In [162]:
import pandas as pd

In [182]:
class CSVGenerator(object):
    def __init__(self):
        self.all_ids = []
        self.csv_data = {}
    
    def fillData(self, lis):
        csv_data = {}
        _profiles = []
        _courses = []
        _discipline = []
        _time = []
        for pid in self.all_ids:
            pobj = all_profile_info[pid]
            for edu in pobj.all_educations:
                _profiles.append(Helper.get_profile_id_from_linkedin_url(pid))
                _courses.append(edu['Course'])
                _discipline.append(edu['Discipline'])
                _time.append(edu['Time Period'])
        csv_data['ProfileID'] = _profiles
        csv_data['Course'] = _courses
        csv_data['Discipline'] = _discipline
        csv_data['Time Period'] = _time
        return csv_data
    
    def saveXLSX(self, lis=None, name = 'out.xlsx'):
        if lis is None:
            self.all_ids = list(all_profile_info.keys())
            self.csv_data = self.fillData(self.all_ids)
        
        df = pd.DataFrame(self.csv_data)
        df.to_excel(name)

In [183]:
generator = CSVGenerator()
generator.saveXLSX()

In [97]:
lis = filtered_profiles.getFiltered(student = True)
print(lis)
lis = ['/in/ayushraj0309/']
my_profile.sendAnnouncement(lis, 'Hi!', 'Please accept my request')

['/in/anupam-agrawal-a822b011/', '/in/abhi4gupta/', '/in/prateek-m-058795114/', '/in/anish-goswami-49a3b7185/', '/in/pranal-maheshwari-31a850192/', '/in/abhigyansharma29/', '/in/blackdchase/', '/in/raunak-rathour-84a7b71aa/', '/in/utkarsh--priyam/', '/in/ayushraj0309/']
Request Pending


In [99]:
print(all_profile_info[lis[0]].all_educations)

[{'URL ID': 'https://www.linkedin.com/company/497317/', 'Insititution': 'Indian Institute Of Information Technology', 'Course': "Bachelor's degree", 'Discipline': ' Information Technology', 'Time Period': '2018 - 2022'}]


Text

In [4]:
import ipywidgets as widgets

In [5]:
widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

IntSlider(value=3, description='Slider:', max=10)

In [6]:
from IPython.display import display
slider = widgets.IntSlider()
display(slider)

IntSlider(value=0)

In [7]:
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description='Hello')
display(button)

Button(description='Hello', style=ButtonStyle())

In [45]:
v = 'Hell'
ALL = 'ALL'

import pandas as pd
import numpy as np
df = pd.read_csv('test_linkedin.csv')

In [46]:
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [47]:
df

,Name,Year,Course,PID
0,Abhishek,2018,Bachelor of Technology - Btech,/in/abhi4gupta/
1,Ayush,2019,B.tech,/in/ayushraj0309/


In [57]:

# dropdown_year = widgets.Dropdown(options = ['2018','2019'])
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df.Year))

output_year = widgets.Output() 

def dropdown_year_eventhandler(change):
    print('asfgdhfbjnfdjcv')
    # output_year.clear_output()
    print(df.Year, output_year)
    if (change.new == ALL):
        display(df)
    else:
        display(df[df.Year == change.new])

dropdown_year.observe(dropdown_year_eventhandler, names='value')


display(dropdown_year)

Dropdown(options=('ALL', 2018, 2019), value='ALL')

In [59]:
print(v)
x = 'X'

Hell


In [63]:
w = widgets.Dropdown(
    options=['Addition', 'Multiplication', 'Subtraction', 'Division'],
    value='Addition',
    description='Task:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])
    x = "changed to %s" % change['new']

w.observe(on_change)

display(w)

Dropdown(description='Task:', options=('Addition', 'Multiplication', 'Subtraction', 'Division'), value='Additi…

In [90]:
_course = ''
_year = ''

In [116]:
df[df.Year == 2019]

,Name,Year,Course,PID
1,Ayush,2019,B.tech,/in/ayushraj0309/


In [161]:
from ipywidgets import widgets 
lbl1=widgets.Label('Course') 
display(lbl1) 
text1=widgets.Text() 
display(text1) 
lbl2=widgets.Label('Year') 
display(lbl2) 
text2=widgets.Text() 
display(text2) 
btn=widgets.Button(description="Get filtered csv") 
display(btn) 
lbl3=widgets.Label()
# display(df)
display(lbl3)
def filter(b):
    _course = text1.value
    _year = text2.value
    print('l1'+text1.value+text2.value)
    print(_course+_year)
    x=text1.value
    y=text2.value
    # df = df[df.Year == y]
    # display(df)
    df2 = df
    if  x!= '':
        df2 = df[df.Year == int(y)]
    if y != '':
        df2 = df[df.Year == int(y)]
    # display(df2)
    # df = df[df.Year == int(y)]
    pid_str =  
    lbl3.value = str(df2.PID)
    # display(df[df.Year == _year])
    # lbl3.value="result= "+str(x+y)
    # lbl3.value=str(x+y)
btn.on_click(filter)

Label(value='Course')

Text(value='')

Label(value='Year')

Text(value='')

Button(description='Get filtered csv', style=ButtonStyle())

Label(value='')

In [93]:
print(_course, _year)

In [125]:
def getFormattedDFStr(df):
    ans = ''
    for d in df:
        print(d)
    return ans
getFormattedDFStr(df)

Name
Year
Course
PID


''